In [1]:
# установка и импорт библиотек
!pip install -q tensorflow==2.3
!pip install albumentations -q
!pip install pymystem3
#! pip install pymorphy2
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import random
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import sys
import PIL
import cv2
import re

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# keras
import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import *
import albumentations

import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from tqdm import tqdm
#import pymorphy2
from string import punctuation
from pymystem3 import Mystem
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('stopwords')
nltk.download('wordnet')

# plt
import matplotlib.pyplot as plt
import seaborn as sns
#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# ПРОЕКТ CAR PRICE

* Шаг 1. Построим «наивную» ML-модель на табличных данных, которая предсказывает цену по модели и году выпуска. Это позволит нам задать направление для дальнейших экспериментов. Впоследствии на этом этапе вы можете доработать вашу модель из проекта «Выбираем авто выгодно».
* Шаг 2. Обработаем и отнормируем признаки и сделаем первую модель на основе градиентного бустинга с помощью CatBoost.
* Шаг 3. Решим эту же задачу с помощью DL (модель NN Tabular) и сравним результаты.
* Шаг 4. Добавим текстовые данные (NLP) и сделаем Multi-Input нейронную сеть для анализа и табличных данных, и текста одновременно.
* Шаг 5. Добавим обработку изображений в Multi-Input нейронную сеть.
* Шаг 6. Осуществим ансамблирование градиентного бустинга и нейронной сети (усредним их предсказания).

![](https://a.d-cd.net/OSOIkXrRNf4leu6chVWxvcHB2M8-960.jpg)

In [2]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Tensorflow   :', tf.__version__)

In [3]:
# фиксируем значения
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
x_size = 8
y_size = 5

In [4]:
def mape(y_true, y_pred):
    """функция для определения целевой метрики"""
    return np.mean(np.abs((y_pred-y_true)/y_true))

def getCountPlot(df, feature, x_s=x_size, y_s=y_size):
    """функция для построения каунтпловот для категорийных признаков"""
    plt.figure(figsize=(x_s, y_s))
    sns.countplot(data=df,
            order = df[feature].value_counts().index, 
            y=feature)
    plt.title(f'Распределение значений признака {feature}')
    plt.show;
    
def emission_replacement(df, column, method='median'):
    '''
    Заменяет выбросы в серии вычисленным значением 
    method='median' - замена на медиану
    method='average' - замена на среднее
    method ='probable' - вероятностное распределение
    '''
    IQR = df[column].quantile(0.75) - df[column].quantile(0.25)
    perc25 = df[column].quantile(0.25)
    perc75 = df[column].quantile(0.75)

    f = perc25 - 1.5*IQR
    l = perc75 + 1.5*IQR

    if method =='median':
        df.loc[(df[column] < f) | (df[column] > l), column] = df[column].median()
    elif method =='average':
        df.loc[(df[column] < f) | (df[column] > l), column] = df[column].mean()
    elif method =='probable':
        # замена выбросов на nan 
        df[column] = np.where((df[column] < f) | (df[column] > l), np.nan, df[column])
        mask = df[column].isna()
        # статистика распределения значений
        p = df[column].value_counts() / len(df[column].dropna())
        # заполняем пропуски с вероятностью `p`
        df.loc[mask, column] = np.random.choice(p.index.to_list(),
                                            size=mask.sum(), 
                                            p=p.to_list())

In [5]:
!pip freeze > requirements.txt

# DATA

типы признаков:

* bodyType - категориальный
* brand - категориальный
* color - категориальный
* description - текстовый
* engineDisplacement - числовой, представленный как текст
* enginePower - числовой, представленный как текст
* fuelType - категориальный
* mileage - числовой
* modelDate - числовой
* model_info - категориальный
* name - категориальный, желательно сократить размерность
* numberOfDoors - категориальный
* price - числовой, целевой
* productionDate - числовой
* sell_id - изображение (файл доступен по адресу, основанному на sell_id)
* vehicleConfiguration - не используется (комбинация других столбцов)
* vehicleTransmission - категориальный
* Владельцы - категориальный
* Владение - числовой, представленный как текст
* ПТС - категориальный
* Привод - категориальный
* Руль - категориальный

In [6]:
# загружаем данные 
DATA_DIR = '../input/sf-dst-car-price-prediction-part2/'
train = pd.read_csv(DATA_DIR + 'train.csv')
test = pd.read_csv(DATA_DIR + 'test.csv')
sample_submission = pd.read_csv(DATA_DIR + 'sample_submission.csv')

# Model 1: Создадим "наивную" модель 
Эта модель будет предсказывать среднюю цену по модели и году выпуска. 
C ней будем сравнивать другие модели.



In [7]:
# split данных
data_train, data_test = train_test_split(train, test_size=0.15, shuffle=True, random_state=RANDOM_SEED)

In [8]:
# Наивная модель
predicts = []
for index, row in pd.DataFrame(data_test[['model_info', 'productionDate']]).iterrows():
    query = f"model_info == '{row[0]}' and productionDate == '{row[1]}'"
    predicts.append(data_train.query(query)['price'].median())

# заполним не найденные совпадения
predicts = pd.DataFrame(predicts)
predicts = predicts.fillna(predicts.median())

# округлим
predicts = (predicts // 1000) * 1000

#оцениваем точность
print(f"Точность наивной модели по метрике MAPE: {(mape(data_test['price'], predicts.values[:, 0]))*100:0.2f}%")

Точность наивной модели по метрике MAPE: 19.88%

# EDA and preproc

In [9]:
train['sample'] = 1 # помечаем где у нас трейн
test['sample'] = 0 # помечаем где у нас тест
test['price'] = 0 # в тесте у нас нет значения price, мы его должны предсказать, поэтому пока просто заполняем нулями

data = test.append(train, sort=False).reset_index(drop=True) # объединяем
print(train.shape, test.shape, data.shape)

In [10]:
data.info()

In [11]:
# взглянем на количество пропусков
data.isnull().sum()

In [12]:
# проверим на дубликаты
len(data.drop_duplicates()) - len(data)

## Осмотрим признаки

In [13]:
# bodyType
getCountPlot(data,'bodyType', 5, 3)

In [14]:
# brand
getCountPlot (data,'brand', 4, 2)

In [15]:
# color
getCountPlot(data,'color', 5, 4)

In [16]:
# engineDisplacement
getCountPlot(data,'engineDisplacement', 8, 12)

In [17]:
# извлечём цифры
data['engineDisplacement'] = data['engineDisplacement'].astype(str).apply(lambda x: x.split()[0])
# заменим неопределённость на число
data['engineDisplacement'] =  data['engineDisplacement'].replace('undefined', 0.0)
# приведём всё к одному формату
data['engineDisplacement'] = data['engineDisplacement'].astype(float)

In [18]:
# enginePower
data['enginePower'].unique()

In [19]:
# извлечём цифры мощности и приведём к удобному формату
data['enginePower'] = data['enginePower'].str.split().apply(lambda x: x[0]) 
data['enginePower'] = data['enginePower'].apply(lambda x: int(x))

In [20]:
# fuelType
getCountPlot(data,'fuelType', 5, 2)

In [21]:
# model_info
data['model_info'].unique()

In [22]:
# name
data['name'].sample(10)

In [23]:
# большинство характеристик упоминаются в других столбцах
# вынесим отсюда пару признаков
data['4wd'] = data['name'].apply(lambda x: 1 if '4WD' in x else 0)
data['xdrive'] = data['name'].apply(lambda x: 1 if 'xDrive' in x else 0)

In [24]:
# и удалим
data.drop(['name'], axis=1, inplace=True)

In [25]:
# numberOfDoors
getCountPlot(data,'numberOfDoors', 4, 2)

In [26]:
# vehicleConfiguration
data['vehicleConfiguration'].value_counts()

In [27]:
# все параметры упоминаются в других столбцах. удалим
data.drop(['vehicleConfiguration'], axis=1, inplace=True)

In [28]:
# vehicleTransmission
getCountPlot(data,'vehicleTransmission', 4.2, 2)

In [29]:
# Владельцы
getCountPlot(data,'Владельцы', 4, 2)

In [30]:
# Владение
data['Владение'].value_counts()

In [31]:
# больше половины значений пустые. удалим
data.drop(['Владение'], axis=1, inplace=True)

In [32]:
# ПТС
getCountPlot(data,'ПТС', 4, 2)

In [33]:
# Руль
getCountPlot(data,'Руль', 4, 2)

In [34]:
# нет вариативности признака. удалим
data.drop(['Руль'], axis=1, inplace=True)

In [35]:
# Привод
getCountPlot(data,'Привод', 4, 2)

In [36]:
#посмотрим, как выглядят распределения числовых признаков
def visualize_distributions(titles_values_dict):
  columns = min(3, len(titles_values_dict))
  rows = (len(titles_values_dict) - 1) // columns + 1
  fig = plt.figure(figsize = (columns * 5, rows * 3))
  for i, (title, values) in enumerate(titles_values_dict.items()):
    hist, bins = np.histogram(values, bins = 20)
    ax = fig.add_subplot(rows, columns, i + 1)
    ax.bar(bins[:-1], hist, width = (bins[1] - bins[0]) * 0.7)
    ax.set_title(title)
  plt.show()

visualize_distributions({
    'mileage': train['mileage'].dropna(),
    'modelDate': train['modelDate'].dropna(),
    'productionDate': train['productionDate'].dropna()
})

In [37]:
# избавимся от выбросов
emission_replacement(data, 'mileage', method='median')

In [38]:
# добавим призанки 
# лет в использовании
data['years_in_use'] = 2021 - data['productionDate']
# количество пробега за год
data['mileage_per_year'] = data['mileage'] / data['years_in_use']

In [39]:
#создали список категориальных
categorical_features = ['bodyType', 'brand', 'color', 'fuelType', 'model_info', '4wd', 'xdrive', 'numberOfDoors',
                        'vehicleTransmission', 'Владельцы', 'ПТС', 'Привод']

#cоздали список числовых признаков
numerical_features = ['productionDate','modelDate', 'mileage',
                     'years_in_use', 'mileage_per_year', 'enginePower', 'engineDisplacement']

In [40]:
#логарифмируем числовые признаки

for col in data[numerical_features]:
    data[col] = data[col].apply(lambda x: np.log(x) if x>0 else x)

In [41]:
def preproc_data(df_input):
    '''функция для удаления лишний признаков, энкодингов'''
    
    df_output = df_input.copy()
    
    # убираем не нужные для модели признаки
    df_output.drop(['description','sell_id'], axis = 1, inplace=True)
    
     
    # Далее заполняем пропуски
    for column in numerical_features:
        df_output[column].fillna(df_output[column].median(), inplace=True)

    
    # Нормализация данных
    scaler = MinMaxScaler()
    for column in numerical_features:
        df_output[column] = scaler.fit_transform(df_output[[column]])[:,0]
    
    # Label Encoding
    for column in categorical_features:
        df_output[column] = df_output[column].astype('category').cat.codes
        
    # One-Hot Encoding
    df_output = pd.get_dummies(df_output, columns=categorical_features, dummy_na=False)
    
    
    return df_output

# Запускаем и проверяем, что получилось
df_preproc = preproc_data(data)
df_preproc.sample(10)

In [42]:
# логарифмируем целевую переменную
df_preproc['price'] = data['price'].apply(lambda x: np.log(x) if x>0 else x)

## Split data

In [43]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.price.values     # наш таргет
X = train_data.drop(['price'], axis=1)
X_sub = test_data.drop(['price'], axis=1)

### CatBoostRegressor

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, shuffle=True, random_state=RANDOM_SEED)

После нескольких экспериментов оставил learning rate=0.05, дает лучее значение метрики

In [45]:
model = CatBoostRegressor(iterations = 5000,
                          #depth=10,
                          learning_rate = 0.05,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['RMSE', 'MAE'],
                          od_wait=500,
                          #task_type='GPU',
                         )
model.fit(X_train, y_train,
         eval_set=(X_test, y_test),
         verbose_eval=100,
         use_best_model=True,
         #plot=True
         )

In [46]:
test_predict_catboost = model.predict(X_test)
print(f"TEST mape: {(mape(np.exp(y_test), np.exp(test_predict_catboost)))*100:0.2f}%")

TEST mape: TEST mape: 11.03%

### Submission

In [47]:
sub_predict_catboost = model.predict(X_sub)
sample_submission['price'] = np.exp(sub_predict_catboost)
sample_submission.to_csv('catboost_submission.csv', index=False)

# Tabular NN

Построим обычную сеть:

In [48]:
X_train.head(5)

## Simple Dense NN

In [49]:
model = Sequential()
model.add(L.Dense(512, input_dim=X_train.shape[1], activation="relu"))
model.add(L.Dropout(0.5))
model.add(L.Dense(256, activation="relu"))
model.add(L.Dropout(0.5))
model.add(L.Dense(1, activation="linear"))

In [50]:
# model.summary()

In [51]:
# Compile model
optimizer = tf.keras.optimizers.Adam(0.01)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [52]:
checkpoint = ModelCheckpoint('../working/best_model.hdf5' ,save_best_only=True, monitor='val_MAPE', verbose=0  , mode='min')
earlystop = EarlyStopping(monitor='val_MAPE', patience=50, restore_best_weights=True,)
callbacks_list = [checkpoint, earlystop]

In [53]:
history = model.fit(X_train, y_train,
                    batch_size=256,
                    epochs=500, # фактически мы обучаем пока EarlyStopping не остановит обучение
                    validation_data=(X_test, y_test),
                    callbacks=callbacks_list,
                    verbose=0,
                   )

In [54]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.legend()
plt.show();

In [55]:
model.load_weights('../working/best_model.hdf5')
model.save('../working/nn_1.hdf5')

In [56]:
test_predict_nn1 = model.predict(X_test)
print(f"TEST mape: {(mape(np.exp(y_test), np.exp(test_predict_nn1[:,0])))*100:0.2f}%")

TEST mape: 13.49%

# Проведем эксперимент с простой нейронной сетью, добавим батчнормализацию в модель

In [57]:
from tensorflow.keras.layers import BatchNormalization

In [58]:
import tensorflow.keras.backend as K

In [59]:
K.clear_session() # очищаем сессию keras

In [60]:
model = Sequential()
model.add(L.Dense(512, input_dim=X_train.shape[1], activation="relu"))
model.add(BatchNormalization())
model.add(L.Dropout(0.5))
model.add(L.Dense(256, activation="relu"))
model.add(BatchNormalization())
model.add(L.Dropout(0.5))
model.add(L.Dense(1, activation="linear"))

In [61]:
model.summary()

In [62]:
# Compile model
optimizer = tf.keras.optimizers.Adam(0.01)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [63]:
checkpoint = ModelCheckpoint('../working/best_model.hdf5' ,save_best_only=True, monitor='val_MAPE', verbose=0  , mode='min')
earlystop = EarlyStopping(monitor='val_MAPE', patience=50, restore_best_weights=True,)
callbacks_list = [checkpoint, earlystop]


In [64]:
history = model.fit(X_train, y_train,
                    batch_size=256,
                    epochs=500, # фактически мы обучаем пока EarlyStopping не остановит обучение
                    validation_data=(X_test, y_test),
                    callbacks=callbacks_list,
                    verbose=0,
                   )

In [65]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.legend()
plt.show();

In [66]:
model.load_weights('../working/best_model.hdf5')
model.save('../working/nn_1.hdf5')

In [67]:
test_predict_nn1 = model.predict(X_test)
print(f"TEST mape: {(mape(np.exp(y_test), np.exp(test_predict_nn1[:,0])))*100:0.2f}%")

TEST mape: 11.07%

Добавление батчнормализации позволило улучшить метрику с 14% до 12%

Попробуем поработать с learning rate:

In [68]:
from tensorflow.keras import regularizers

In [69]:
K.clear_session() # очищаем сессию keras

In [70]:
model = Sequential()
model.add(L.Dense(512, input_dim=X_train.shape[1],  kernel_regularizer=regularizers.l1_l2(l1=0.000000001, 
        l2=0.000000001), activation="relu"))
model.add(BatchNormalization())
model.add(L.Dropout(0.5))
model.add(L.Dense(256,kernel_regularizer=regularizers.l1_l2(l1=0.000000001, 
        l2=0.000000001), activation="relu"))
model.add(BatchNormalization())
model.add(L.Dropout(0.5))

model.add(L.Dense(1, activation="linear"))

In [71]:
model.summary()

In [72]:
# Compile model
optimizer = tf.keras.optimizers.Adam(0.01)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [73]:
checkpoint = ModelCheckpoint('../working/best_model.hdf5' , monitor='val_MAPE',save_best_only=True, verbose=0  , mode='min')
earlystop = EarlyStopping(monitor='val_MAPE', patience=50, restore_best_weights=True,)
callbacks_list = [checkpoint, earlystop]

In [74]:
history = model.fit(X_train, y_train,
                    batch_size=256,
                    epochs=500, # фактически мы обучаем пока EarlyStopping не остановит обучение
                    validation_data=(X_test, y_test),
                    callbacks=callbacks_list,
                    verbose=0,
                   )

In [75]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.legend()
plt.show();

In [76]:
model.load_weights('../working/best_model.hdf5')
model.save('../working/nn_1.hdf5')

In [77]:
test_predict_nn1 = model.predict(X_test)
print(f"TEST mape: {(mape(np.exp(y_test), np.exp(test_predict_nn1[:,0])))*100:0.2f}%")

Четкой разницы не видно, эксперимент не удался, но по идее можно еще попробовать подобрать LR таким образом чтобы улучшить метрику

Объединим предсказания нейронной сети и catboost путем усреднения:

In [78]:
# попробуем объединить предсказания catboost и нейронной сети
new_pred = (test_predict_catboost + test_predict_nn1[:,0])/2
print(f"TEST mape: {(mape(np.exp(y_test), np.exp(new_pred)))*100:0.2f}%")

TEST mape: 10.93%

Очень даже неплохо, минимум трудозатрат, а метрика улучшилась, сделаем сабмит этой парочки

In [79]:
# сабмит для нейронной сети отдельно
sub_predict_nn1 = model.predict(X_sub)
sample_submission['price'] = np.exp(sub_predict_nn1[:,0])
sample_submission.to_csv('nn1_submission.csv', index=False)

In [80]:
sample_submission['price'] = (np.exp(sub_predict_nn1[:,0])+np.exp(sub_predict_catboost))/2
sample_submission.to_csv('nn1+cat_submission.csv', index=False)

В итоге комбинация Catboost и простой нейросети улучшили результаты на каггле с 12.4 до 12.3, неплохо)

# Model 4: NLP + Multiple Inputs

In [81]:
data.description

In [82]:
# TOKENIZER
# максимальное количество слов использовано
MAX_WORDS = 100000
# максимальное количество слов в строке
MAX_SEQUENCE_LENGTH = 256

In [83]:
# split данных
text_train = data.description.iloc[X_train.index]
text_test = data.description.iloc[X_test.index]
text_sub = data.description.iloc[X_sub.index]

In [84]:
%%time
tokenize = Tokenizer(num_words=MAX_WORDS)
tokenize.fit_on_texts(data.description)

In [85]:
tokenize.word_index

In [86]:
%%time
text_train_sequences = sequence.pad_sequences(tokenize.texts_to_sequences(text_train), maxlen=MAX_SEQUENCE_LENGTH)
text_test_sequences = sequence.pad_sequences(tokenize.texts_to_sequences(text_test), maxlen=MAX_SEQUENCE_LENGTH)
text_sub_sequences = sequence.pad_sequences(tokenize.texts_to_sequences(text_sub), maxlen=MAX_SEQUENCE_LENGTH)

print(text_train_sequences.shape, text_test_sequences.shape, text_sub_sequences.shape, )

In [87]:
text_train.iloc[0]

In [88]:
print(text_train.iloc[6])
print(text_train_sequences[6])

Построим сеть с двумя входами: сеть обучающаяся на текстовой информации и сеть обучающаяся на наших табличных значениях.

In [89]:
K.clear_session() # очищаем сессию keras

In [90]:
model_nlp = Sequential()
model_nlp.add(L.Input(shape=MAX_SEQUENCE_LENGTH, name="seq_description"))
model_nlp.add(L.Embedding(len(tokenize.word_index)+1, MAX_SEQUENCE_LENGTH,))
model_nlp.add(L.LSTM(256, return_sequences=True))
model_nlp.add(L.Dropout(0.5))
model_nlp.add(L.LSTM(128,))
model_nlp.add(L.Dropout(0.25))
model_nlp.add(L.Dense(64, activation="relu"))
model_nlp.add(L.Dropout(0.25))

In [91]:
model_mlp = Sequential()
model_mlp.add(L.Dense(512, input_dim=X_train.shape[1], activation="relu"))
model_mlp.add(L.Dropout(0.5))
model_mlp.add(L.Dense(256, activation="relu"))
model_mlp.add(L.Dropout(0.5))

Общая голова, принимает на вход выходы первых двух сетей

In [92]:
combinedInput = L.concatenate([model_nlp.output, model_mlp.output])
# being our regression head
head = L.Dense(64, activation="relu")(combinedInput)
head = L.Dense(1, activation="linear")(head)

model = Model(inputs=[model_nlp.input, model_mlp.input], outputs=head)

In [93]:
model.summary()

In [94]:
optimizer = tf.keras.optimizers.Adam(0.01)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [95]:
checkpoint = ModelCheckpoint('../working/best_model.hdf5', monitor=['val_MAPE'], verbose=0, mode='min')
earlystop = EarlyStopping(monitor='val_MAPE', patience=10, restore_best_weights=True,)
callbacks_list = [checkpoint, earlystop]

In [96]:
history = model.fit([text_train_sequences, X_train], np.exp(y_train),
                    batch_size=512,
                    epochs=500, # фактически мы обучаем пока EarlyStopping не остановит обучение
                    validation_data=([text_test_sequences, X_test],np.exp(y_test)),
                    callbacks=callbacks_list
                   )


In [97]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.legend()
plt.show();

In [98]:
test_predict_nn2 = model.predict([text_test_sequences, X_test])
print(f"TEST mape: {(mape(np.exp(y_test), test_predict_nn2[:,0]))*100:0.2f}%")

TEST mape: 12.02%

 Сложная сеть дала на выходе неплохую метрику.Так же сделаем сабмит из объединения результатов для этой сети  и catboost:

In [99]:
sub_predict_nn2 = model.predict([text_sub_sequences,X_sub])

In [100]:
sub_predict_nn2 = model.predict([text_sub_sequences,X_sub])
sample_submission['price'] = (sub_predict_nn2[:,0]+np.exp(sub_predict_catboost))/2

In [101]:
sample_submission.to_csv('nn2+cat_submission.csv', index=False)

нейросеть, с обработкой текста + простая нейросеть и catboost в итоге дали пока что лучшую точность на каггле 11.797!

Эксперимент берем исправленную мной нейросеть, и текстовую, + результаты catboost

In [102]:
K.clear_session() # очищаем сессию keras

In [103]:
# TOKENIZER
# максимальное количество слов использовано
MAX_WORDS = 100000
# максимальное количество слов в строке
MAX_SEQUENCE_LENGTH = 256

In [104]:
# split данных
text_train = data.description.iloc[X_train.index]
text_test = data.description.iloc[X_test.index]
text_sub = data.description.iloc[X_sub.index]

In [105]:
%%time
tokenize = Tokenizer(num_words=MAX_WORDS)
tokenize.fit_on_texts(data.description)

In [106]:
%%time
text_train_sequences = sequence.pad_sequences(tokenize.texts_to_sequences(text_train), maxlen=MAX_SEQUENCE_LENGTH)
text_test_sequences = sequence.pad_sequences(tokenize.texts_to_sequences(text_test), maxlen=MAX_SEQUENCE_LENGTH)
text_sub_sequences = sequence.pad_sequences(tokenize.texts_to_sequences(text_sub), maxlen=MAX_SEQUENCE_LENGTH)

print(text_train_sequences.shape, text_test_sequences.shape, text_sub_sequences.shape, )

In [107]:
model_nlp = Sequential()
model_nlp.add(L.Input(shape=MAX_SEQUENCE_LENGTH, name="seq_description"))
model_nlp.add(L.Embedding(len(tokenize.word_index)+1, MAX_SEQUENCE_LENGTH,))
model_nlp.add(L.LSTM(256, return_sequences=True))
model_nlp.add(L.Dropout(0.5))
model_nlp.add(L.LSTM(128,))
model_nlp.add(L.Dropout(0.25))
model_nlp.add(L.Dense(64, activation="relu"))
model_nlp.add(L.Dropout(0.25))

In [108]:
model = Sequential()
model.add(L.Dense(512, input_dim=X_train.shape[1], activation="relu"))
model.add(BatchNormalization())
model.add(L.Dropout(0.5))
model.add(L.Dense(256, activation="relu"))
model.add(BatchNormalization())
model.add(L.Dropout(0.5))
model.add(L.Dense(1, activation="linear"))

In [109]:
combinedInput = L.concatenate([model_nlp.output, model_mlp.output])
# being our regression head
head = L.Dense(64, activation="relu")(combinedInput)
head = L.Dense(1, activation="linear")(head)

model = Model(inputs=[model_nlp.input, model_mlp.input], outputs=head)

In [110]:
optimizer = tf.keras.optimizers.Adam(0.01)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [111]:
checkpoint = ModelCheckpoint('../working/best_model.hdf5', monitor=['val_MAPE'], verbose=0, mode='min')
earlystop = EarlyStopping(monitor='val_MAPE', patience=10, restore_best_weights=True,)
callbacks_list = [checkpoint, earlystop]

In [112]:
history = model.fit([text_train_sequences, X_train], np.exp(y_train),
                    batch_size=512,
                    epochs=500, # фактически мы обучаем пока EarlyStopping не остановит обучение
                    validation_data=([text_test_sequences, X_test],np.exp(y_test)),
                    callbacks=callbacks_list
                   )

In [113]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.legend()
plt.show();

In [114]:
test_predict_nn2 = model.predict([text_test_sequences, X_test])
print(f"TEST mape: {(mape(np.exp(y_test), test_predict_nn2[:,0]))*100:0.2f}%")

TEST mape: 12.28%

Попробуем засабмитить результат с усредниением catboost

In [115]:
sub_predict_nn2 = model.predict([text_sub_sequences,X_sub])
sample_submission['price'] = (sub_predict_nn2[:,0]+np.exp(sub_predict_catboost))/2

In [116]:
sample_submission.to_csv('nn2_exp+cat_submission.csv', index=False)

Это лучшая точность на kaggle 11.755!

### Добавляем картинки

In [117]:
# убедимся, что цены и фото подгрузились верно
plt.figure(figsize = (12,8))

random_image = train.sample(n = 9)
random_image_paths = random_image['sell_id'].values
random_image_cat = random_image['price'].values

for index, path in enumerate(random_image_paths):
    im = PIL.Image.open(DATA_DIR+'img/img/' + str(path) + '.jpg')
    plt.subplot(3, 3, index + 1)
    plt.imshow(im)
    plt.title('price: ' + str(random_image_cat[index]))
    plt.axis('off')
plt.show()

In [118]:
size = (320, 240)

def get_image_array(index):
    images_train = []
    for index, sell_id in enumerate(data['sell_id'].iloc[index].values):
        image = cv2.imread(DATA_DIR + 'img/img/' + str(sell_id) + '.jpg')
        assert(image is not None)
        image = cv2.resize(image, size)
        images_train.append(image)
    images_train = np.array(images_train)
    print('images shape', images_train.shape, 'dtype', images_train.dtype)
    return(images_train)

images_train = get_image_array(X_train.index)
images_test = get_image_array(X_test.index)
images_sub = get_image_array(X_sub.index)

In [119]:
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose
)


#пример взят из официальной документации: https://albumentations.readthedocs.io/en/latest/examples.html
augmentation = Compose([
   # HorizontalFlip(),
    OneOf([
        IAAAdditiveGaussianNoise(),
        GaussNoise(),
    ], p=0.2),
    OneOf([
        MotionBlur(p=0.2),
        MedianBlur(blur_limit=3, p=0.1),
        Blur(blur_limit=3, p=0.1),
    ], p=0.2),
    ShiftScaleRotate(shift_limit=0.1, scale_limit=0.02, rotate_limit=5, p=1),
    OneOf([
        OpticalDistortion(p=0.3),
        GridDistortion(p=0.1),
        IAAPiecewiseAffine(p=0.3),
    ], p=0.2),
    OneOf([
        CLAHE(clip_limit=2),
        IAASharpen(),
        IAAEmboss(),
        RandomBrightnessContrast(),
    ], p=0.3),
    HueSaturationValue(p=0.3),
], p=1)

#пример
plt.figure(figsize = (12,8))
for i in range(9):
    img = augmentation(image = images_train[0])['image']
    plt.subplot(3, 3, i + 1)
    plt.imshow(img)
    plt.axis('off')
plt.show()

In [120]:
def make_augmentations(images):
    print('применение аугментаций', end = '')
    augmented_images = np.empty(images.shape)
    for i in range(images.shape[0]):
        if i % 200 == 0:
            print('.', end = '')
        augment_dict = augmentation(image = images[i])
        augmented_image = augment_dict['image']
        augmented_images[i] = augmented_image
    print('')
    return augmented_images

In [121]:
# TOKENIZER
# максимальное количество слов использовано
MAX_WORDS = 100000
# максимальное количество слов в строке
MAX_SEQUENCE_LENGTH = 256

# Лемматизируем и удалим стоп слова
mystem = Mystem() 
russian_stopwords = stopwords.words("russian")

def preprocess_text(text):
    """функция для предобработки текста"""
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
             and token != " " \
             and token.strip() not in punctuation]
    text = " ".join(tokens)

    return text

#df_preproc['description_preproc'] = data['description'].apply(preprocess_text)
df_preproc['description_preproc'] = data['description']


# split данных
#text_train = df_preproc.description_preproc.iloc[X_train.index]
#text_test = df_preproc.description_preproc.iloc[X_test.index]
#text_sub = df_preproc.description_preproc.iloc[X_sub.index]

#%%time
tokenize = Tokenizer(num_words=MAX_WORDS)
tokenize.fit_on_texts(df_preproc.description_preproc)
#tokenize.word_index



Качество модели после препроцессинга description, стало хуже

In [122]:
def process_image(image):
    return augmentation(image = image.numpy())['image']

def tokenize_(descriptions):
    return sequence.pad_sequences(tokenize.texts_to_sequences(descriptions), maxlen = MAX_SEQUENCE_LENGTH)

def tokenize_text(text):
    return tokenize_([text.numpy().decode('utf-8')])[0]

def tf_process_train_dataset_element(image, table_data, text, price):
    im_shape = image.shape
    [image,] = tf.py_function(process_image, [image], [tf.uint8])
    image.set_shape(im_shape)
    [text,] = tf.py_function(tokenize_text, [text], [tf.int32])
    return (image, table_data, text), price

def tf_process_val_dataset_element(image, table_data, text, price):
    [text,] = tf.py_function(tokenize_text, [text], [tf.int32])
    return (image, table_data, text), price

train_dataset = tf.data.Dataset.from_tensor_slices((
    images_train, X_train, df_preproc['description_preproc'].iloc[X_train.index], np.exp(y_train)
    )).map(tf_process_train_dataset_element)

test_dataset = tf.data.Dataset.from_tensor_slices((
    images_test, X_test, df_preproc['description_preproc'].iloc[X_test.index], np.exp(y_test)
    )).map(tf_process_val_dataset_element)
y_sub = np.zeros(len(X_sub))
sub_dataset = tf.data.Dataset.from_tensor_slices((
    images_sub, X_sub, df_preproc['description_preproc'][X_sub.index], y_sub
    )).map(tf_process_val_dataset_element)

#проверяем, что нет ошибок (не будет выброшено исключение):
train_dataset.__iter__().__next__();
test_dataset.__iter__().__next__();
sub_dataset.__iter__().__next__();

In [123]:
# Строим сверточную сеть для анализа изображений без "головы"
#нормализация включена в состав модели EfficientNetB3, поэтому на вход она принимает данные типа uint8
efficientnet_model = tf.keras.applications.efficientnet.EfficientNetB3(weights = 'imagenet', include_top = False, input_shape = (size[1], size[0], 3))
efficientnet_output = L.GlobalAveragePooling2D()(efficientnet_model.output)

In [124]:
#строим нейросеть для анализа табличных данных
tabular_model = Sequential([
    L.Input(shape = X.shape[1]),
    L.Dense(512, activation = 'relu'),
    L.Dropout(0.5),
    L.Dense(256, activation = 'relu'),
    L.Dropout(0.5),
    ])

In [125]:
# NLP
nlp_model = Sequential([
    L.Input(shape=MAX_SEQUENCE_LENGTH, name="seq_description"),
    L.Embedding(len(tokenize.word_index)+1, MAX_SEQUENCE_LENGTH,),
    L.LSTM(256, return_sequences=True),
    L.Dropout(0.5),
    L.LSTM(128),
    L.Dropout(0.25),
    L.Dense(64),
    ])

In [126]:
#объединяем выходы трех нейросетей
combinedInput = L.concatenate([efficientnet_output, tabular_model.output, nlp_model.output])

# being our regression head
head = L.Dense(256, activation="relu")(combinedInput)
head = L.Dense(1,)(head)

model = Model(inputs=[efficientnet_model.input, tabular_model.input, nlp_model.input], outputs=head)
#model.summary()

In [127]:
optimizer = tf.keras.optimizers.Adam(0.005)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [128]:
checkpoint = ModelCheckpoint('../working/best_model.hdf5', monitor=['val_MAPE'], verbose=0, mode='min')
earlystop = EarlyStopping(monitor='val_MAPE', patience=10, restore_best_weights=True,)
callbacks_list = [checkpoint, earlystop]

In [129]:
history = model.fit(train_dataset.batch(30),
                    epochs=100,
                    validation_data = test_dataset.batch(30),
                    callbacks=callbacks_list)

In [130]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show();

In [131]:
model.load_weights('../working/best_model.hdf5')
model.save('../working/nn_final.hdf5')

In [132]:
test_predict_nn3 = model.predict(test_dataset.batch(30))
print(f"TEST mape: {(mape(np.exp(y_test), test_predict_nn3[:,0]))*100:0.2f}%")

TEST mape: 13.15%  - такой результат удалось получить немного изменив параметры аугментации изображений. А именно: Убрал HorisontalFlip,  изменил параметры ShiftScaleRotate( shift_limit = 0.1/0.0625
                                                                   scale_limit = 0.02/0.2
                                                                   rotate_limit = 5/15)
                                                                   
С настройками baseline TEST mape у этой модели получался 13.84% 

In [133]:
sub_predict_nn3 = model.predict(sub_dataset.batch(30))
sample_submission['price'] = sub_predict_nn3[:,0]
sample_submission.to_csv('nn3_submission.csv', index=False)

Эксперимент Multi-input  сеть: табличные данные+текст+ картинки  с использованием  NNсети с батчнорм

In [134]:
K.clear_session() # очищаем сессию keras

In [135]:
# Строим сверточную сеть для анализа изображений без "головы"
#нормализация включена в состав модели EfficientNetB3, поэтому на вход она принимает данные типа uint8
efficientnet_model = tf.keras.applications.efficientnet.EfficientNetB3(weights = 'imagenet', include_top = False, input_shape = (size[1], size[0], 3))
efficientnet_output = L.GlobalAveragePooling2D()(efficientnet_model.output)

In [136]:
#строим нейросеть для анализа табличных данных
model = Sequential()
model.add(L.Dense(512, input_dim=X_train.shape[1], activation="relu"))
model.add(BatchNormalization())
model.add(L.Dropout(0.5))
model.add(L.Dense(256, activation="relu"))
model.add(BatchNormalization())
model.add(L.Dropout(0.5))
model.add(L.Dense(1, activation="linear"))

In [137]:
# NLP
nlp_model = Sequential([
    L.Input(shape=MAX_SEQUENCE_LENGTH, name="seq_description"),
    L.Embedding(len(tokenize.word_index)+1, MAX_SEQUENCE_LENGTH,),
    L.LSTM(256, return_sequences=True),
    L.Dropout(0.5),
    L.LSTM(128),
    L.Dropout(0.25),
    L.Dense(64),
    ])

In [138]:
#объединяем выходы трех нейросетей
combinedInput = L.concatenate([efficientnet_output, tabular_model.output, nlp_model.output])

# being our regression head
head = L.Dense(256, activation="relu")(combinedInput)
head = L.Dense(1,)(head)

model = Model(inputs=[efficientnet_model.input, tabular_model.input, nlp_model.input], outputs=head)
#model.summary()

In [139]:
optimizer = tf.keras.optimizers.Adam(0.005)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [140]:
checkpoint = ModelCheckpoint('../working/best_model.hdf5', monitor=['val_MAPE'], verbose=0, mode='min')
earlystop = EarlyStopping(monitor='val_MAPE', patience=10, restore_best_weights=True,)
callbacks_list = [checkpoint, earlystop]

In [141]:
history = model.fit(train_dataset.batch(30),
                    epochs=100,
                    validation_data = test_dataset.batch(30),
                    callbacks=callbacks_list)

In [142]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show();

In [143]:
model.load_weights('../working/best_model.hdf5')
model.save('../working/nn_final.hdf5')

In [144]:
test_predict_nn3 = model.predict(test_dataset.batch(30))
print(f"TEST mape: {(mape(np.exp(y_test), test_predict_nn3[:,0]))*100:0.2f}%")

TEST mape: 12.69%

Метрика на тесте несколько улучшилась, но все - таки эта модель по качеству уступает более простой без картинок.

### Blend

In [148]:
blend_predict = (np.exp(test_predict_catboost) + test_predict_nn3[:,0]) / 2
print(f"TEST mape: {(mape(np.exp(y_test), blend_predict))*100:0.2f}%")

TEST mape: 10.84%


In [147]:
blend_sub_predict = (np.exp(sub_predict_catboost) + sub_predict_nn3[:,0]) / 2
sample_submission['price'] = blend_sub_predict
sample_submission.to_csv('blend_submission.csv', index=False)

Метрика  на Kaggle 11.91  тоже несколько у ступает лучшему результату модели с табличными и с текстовыми данными

****Итог****

Выводы: 
в EDA
- Данные очищены от пропусков, пустые значения заменены на медианы
- Признаки с большим количеством пропусков или не имеющие распределения относительно целевой переменной( прим"Руль") удалены
- Сформированы новые признаки feature engeneering
- Данные прологарифмированы и нормированы для подачи более качественного материала в модели обучения

Модели:
- Построена наивная модель, для того  чтобы в дальнейшем понимать куда мы двигаемся (улучшение/ухудшение метрики)
- Обучена модель на основе Catboost. Настроены гиперпараметры, лучше всего себя показал LR = 0.05 MAPE = 11.03%
- Обучена модель на основе простой нейросети, неплохой результат дали эксперименты с добавлением Batchnormalisation, улучшили значение метрики
-Построена Multi-input  сеть: табличные данные+текст. Сеть работающая с табличными данными дополнена батчнормализацией из прошлых экспериментов, это повысило результат комбинации
-Построена Multi-input  сеть: табличные данные+текст+ картинки . Эксперимент пока идет, изменил некоторые параметры аугментации, надо бы еще на вход вместо обычной нейронной сети подать мой вариант
- К моделям, построенным на основе нейросетей применен блендинг Catboost. В итоге это и дало лучшие результваты на сабмитах, лучший сабмит на Kaggle 11.75(Multi-input  сеть: табличные данные+текст+ CatBoost - лучший результат). Таким образом удалось улучшить baseline проекта

Что еще можно было сделать:
 - Думаю если поработать еще над текстовой информацией и подать на обучение нейросети более качественную инфу, то в итоге мы получим лучшую метрику для Multi-input  сеть: табличные данные+текст, потенциал там еще есть, не хваатает мозгов
 - Для модели  Multi-input  сеть: табличные данные+текст+ картинки  требуется очень много времени. Возможно в результате удастся добиться лучших результатов чем по другим, более простым сетям. Думаю стоит еще поработать с аугментацией, при правильных настройках качество еще значительно возрастет. Но картинки оборабатываются очень долго, соответственно время экспреиментов вырастает в разы, а прироста относительно базовой модели мне удалось достичь совсем небольшого.
Общий вывод: при ограниченных сроках лучше строить простые модели и комбинируя их ( в данном проекте использован блендинг) можно в короткие сроки добиваться результатов, сравнимых с работой тяжелых нейросетей.
